In [23]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

### Calling Libraries

In [24]:
# this library is to manipulate browser
from selenium import webdriver

# it allows you to work with differen versions of drivers
# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

In [25]:
pwd

'c:\\Users\\KikeMa\\Documents\\Azu\\diplomado\\python\\Ejercicios clase\\web_scrapping\\code'

In [26]:
### ChromeDriverManager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.maximize_window()

url = 'https://www.bumeran.com.pe'
driver.get(url)

In [27]:
## 2. Task Description

In [28]:
### Aplicando los filtros

url1 = 'https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html'
driver.get(url1)
time.sleep(3)  # dejar que cargue la página

## Suggested Scraping Strategy (Two Stages)
### Desarrollamos Stage 1
Scrape all the job listing URLs based on the filters above.
Navigate across all pages if necessary.

In [21]:
from selenium.webdriver.common.by import By

job_links = []

while True:
    time.sleep(3)  # espera a que cargue la página
    
    # Selecciona los enlaces de ofertas en la página
    jobs = driver.find_elements(By.CSS_SELECTOR, "h2.sc-eGbFzm a") 
    
    for job in jobs:
        link = job.get_attribute("href")
        if link and link.startswith("http"):
            job_links.append(link)

    # Intentar pasar a la siguiente página
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Siguiente']")
        driver.execute_script("arguments[0].click();", next_btn)
    except:
        print("✅ No hay más páginas")
        break

# Eliminar duplicados
job_links = list(set(job_links))
print(f"Se encontraron {len(job_links)} ofertas")


InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x0x5ca8a3+63283]
	GetHandleVerifier [0x0x5ca8e4+63348]
	(No symbol) [0x0x403ca0]
	(No symbol) [0x0x43fb48]
	(No symbol) [0x0x4712d6]
	(No symbol) [0x0x46ceb5]
	(No symbol) [0x0x46c436]
	(No symbol) [0x0x3d5755]
	(No symbol) [0x0x3d5cae]
	(No symbol) [0x0x3d613d]
	GetHandleVerifier [0x0x83bb43+2623955]
	GetHandleVerifier [0x0x836daa+2604090]
	GetHandleVerifier [0x0x5f069a+218410]
	GetHandleVerifier [0x0x5e0ed8+154984]
	GetHandleVerifier [0x0x5e742d+180925]
	(No symbol) [0x0x3d5420]
	(No symbol) [0x0x3d4c36]
	GetHandleVerifier [0x0x9768fc+3913612]
	BaseThreadInitThunk [0x0x758c5d49+25]
	RtlInitializeExceptionChain [0x0x76eed6db+107]
	RtlGetAppContainerNamedObjectPath [0x0x76eed661+561]


In [31]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

job_links = []
wait = WebDriverWait(driver, 100)

while True:
    # Espera a que se carguen los títulos de ofertas
    jobs = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, "//h2//a")))

    
    for job in jobs:
        link = job.get_attribute("href")
        if link and link.startswith("http"):
            job_links.append(link)

    # Intentar pasar a la siguiente página
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Siguiente']")
        driver.execute_script("arguments[0].click();", next_btn)
    except:
        print("✅ No hay más páginas")
        break

# Eliminar duplicados
job_links = list(set(job_links))
print(f"Se encontraron {len(job_links)} ofertas")


✅ No hay más páginas
Se encontraron 4 ofertas


In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

options = Options()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html")

wait = WebDriverWait(driver, 15)
job_links = []

while True:
    # 👇 Scroll progresivo para cargar más empleos
    for i in range(5):  # prueba con 5, si aún da pocos sube a 8 o 10
        driver.execute_script(f"window.scrollTo(0, {i*2000});")
        time.sleep(1.5)

    # Espera que aparezcan los links
    jobs = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, "//h2//a"))
    )

    for job in jobs:
        link = job.get_attribute("href")
        if link and link.startswith("http"):
            job_links.append(link)

    # Intentar pasar a la siguiente página
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Siguiente']")
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(3)
    except:
        print("✅ No hay más páginas")
        break

# Eliminar duplicados
job_links = list(set(job_links))

# Guardar en CSV
df = pd.DataFrame(job_links, columns=["Job_URL"])
df.to_csv("stage1_job_links.csv", index=False, encoding="utf-8-sig")

print(f"Stage 1 completado: {len(job_links)} links guardados en stage1_job_links.csv")


✅ No hay más páginas
Stage 1 completado: 4 links guardados en stage1_job_links.csv
